In [11]:
import sagemaker
import boto3
import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = sess.boto_region_name
role = sagemaker.get_execution_role()
prefix = "sagemaker-datasetprep"

In [12]:
from time import gmtime, strftime
import time

In [8]:
!wget -q https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz

In [9]:
!tar -xzf dbpedia_csv.tar.gz

tar: dbpedia_csv/test.csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/classes.txt: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/train.csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/readme.txt: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: Exiting with failure status due to previous errors


In [15]:
!head dbpedia_csv/train.csv -n 3

1,"E. D. Abbott Ltd"," Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972."
1,"Schwan-Stabilo"," Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss."
1,"Q-workshop"," Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablishe

In [16]:
!grep -i "automatic electric"  dbpedia_csv/train.csv 

1,"Automatic Electric"," Automatic Electric Company (AE) was the largest of the manufacturing units of the Automatic Electric Group. It was a telephone equipment supplier for independent telephone companies in North America and also had a world-wide presence. With its line of automatic telephone exchanges it was also a long-term supplier of switching equipment to the Bell System starting in 1919."
1,"Tokyo Marui"," Tokyo Marui Co. Ltd (株式会社東京マルイ Kabushiki-gaisha Tōkyō Marui) is an airsoft gun manufacturer located in Adachi Tokyo Japan. They are best known for creating the AEG(Automatic electric gun). Its main market is Japan but third-party retailers sell in Hong Kong (PRC) Taiwan (ROC) South Korea East Asia and worldwide. Such is the popularity of its guns that the company has its own center for airsoft sport called Tokyo Marui BB Sports Field."


In [17]:
!cat dbpedia_csv/classes.txt

Company
EducationalInstitution
Artist
Athlete
OfficeHolder
MeanOfTransportation
Building
NaturalPlace
Village
Animal
Plant
Album
Film
WrittenWork


In [18]:
d_label = {}
with open ('dbpedia_csv/classes.txt') as f:
    for i, label in enumerate(f.readlines()):
        d_label[str(i + 1)] = label.strip()
print(d_label)     

{'1': 'Company', '2': 'EducationalInstitution', '3': 'Artist', '4': 'Athlete', '5': 'OfficeHolder', '6': 'MeanOfTransportation', '7': 'Building', '8': 'NaturalPlace', '9': 'Village', '10': 'Animal', '11': 'Plant', '12': 'Album', '13': 'Film', '14': 'WrittenWork'}


In [19]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
def transform_text(row):
    cur_row = []
    label = f'__label__{d_label[row[0]]}'
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

In [21]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv

def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[: int(keep * len(all_rows))]
    pool = Pool(processes = multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_text, all_rows)
    pool.close()
    pool.join()
    
    with open(output_file, 'w') as csvoutfile:
        csvwriter = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csvwriter.writerows(transformed_rows)

In [22]:
%%time
preprocess('dbpedia_csv/train.csv', 'dbpedia_csv/dbpedia.train', keep=0.2)
preprocess('dbpedia_csv/test.csv', 'dbpedia_csv/dbpedia.validation')

CPU times: user 9.91 s, sys: 1.42 s, total: 11.3 s
Wall time: 1min 27s


In [23]:
!head -n 1 dbpedia_csv/dbpedia.train

__label__Company winceyco winceyco is an entertainment group specializing in education in the state of new jersey with an emphasis on musical assembly series for schools and churches . the limited liability company was founded by wincey terry a professional vocalist who has worked with industry notables like spike lee bill cosby tina turner and garrison keilor . the group travels throughout the new jersey region delivering about 100 performances annually .


In [24]:
image=sagemaker.image_uris.retrieve(framework='blazingtext', 
                                    region=region, 
                                    version='1')
print(image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:1


In [25]:
s3_output_location = f's3://{bucket}/{prefix}/output'

estimator = sagemaker.estimator.Estimator(
            image,
            role,
            instance_count=1,
            instance_type='ml.c5.2xlarge',
            volume_size=30,
            max_run=360000,
            input_mode='File',
            enable_sagemaker_metrics=True,
            output_path=s3_output_location,
            hyperparameters={
                'mode': 'supervised',
                'epochs': 20,
                'min_count': 2,
                'learning_rate': 0.05,
                'vector_dim': 10,
                'early_stopping': True,
                'patience': 4,
                'min_epochs': 5,
                'word_ngrams': 2,
            },
)

In [26]:
train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

sess.upload_data(path='dbpedia_csv/dbpedia.train', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='dbpedia_csv/dbpedia.validation', bucket=bucket, key_prefix=validation_channel)

s3_train_data = f's3://{bucket}/{train_channel}'
s3_validation_data = f's3://{bucket}/{validation_channel}'
print(s3_train_data)
print(s3_validation_data)

data_channels = {'train': s3_train_data, 
                 'validation': s3_validation_data}

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'dbpedia-blazingtext-{exp_datetime}'

estimator.fit(inputs=data_channels,
              job_name=jobname,
              logs=True)

s3://sagemaker-us-east-1-104877823522/sagemaker-datasetprep/train
s3://sagemaker-us-east-1-104877823522/sagemaker-datasetprep/validation
2022-04-13 02:57:19 Starting - Starting the training job...
2022-04-13 02:57:49 Starting - Preparing the instances for trainingProfilerReport-1649818639: InProgress
......
2022-04-13 02:58:49 Downloading - Downloading input data...
2022-04-13 02:59:09 Training - Downloading the training image...
2022-04-13 02:59:49 Training - Training image download completed. Training in progress.Arguments: train
[04/13/2022 02:59:41 WARNING 140024490346304] Loggers have already been setup.
[04/13/2022 02:59:41 WARNING 140024490346304] Loggers have already been setup.
[04/13/2022 02:59:41 INFO 140024490346304] nvidia-smi took: 0.025160551071166992 secs to identify 0 gpus
[04/13/2022 02:59:41 INFO 140024490346304] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[04/13/2022 02:59:41 INFO 1400244903463

In [27]:
estimator.model_data

's3://sagemaker-us-east-1-104877823522/sagemaker-datasetprep/output/dbpedia-blazingtext-2022-04-13-02-57-19/output/model.tar.gz'

In [28]:
!aws s3 cp {estimator.model_data} ./dbpedia_csv/

download: s3://sagemaker-us-east-1-104877823522/sagemaker-datasetprep/output/dbpedia-blazingtext-2022-04-13-02-57-19/output/model.tar.gz to dbpedia_csv/model.tar.gz


In [30]:
%%sh
cd dbpedia_csv/
tar -zxf model.tar.gz

In [31]:
!pip install -q sagemaker-experiments

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [48]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError

In [49]:
experiment_name = 'dbpedia-text-classification'

try:
    experiment = Experiment.create(
        experiment_name = experiment_name,
        description = 'Training a text classification using dbpedia dataset.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment')

dbpedia-text-classification experiment already exists! Reusing the existing experiment


In [53]:
for lr in [0.1, 0.01, 0.001]:
    exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
    jobname = f'dbpedia-blazingtext-{exp_datetime}'
    
    # Create a new trial for the experiment
    exp_trial = Trial.create(
        experiment_name = experiment_name,
        trial_name = jobname)
    
    experiment_config = {
        'ExperimentName':experiment_name,
        'TrialName': exp_trial.trial_name,
        'TrialComponentDisplayName': 'Training'}
    
    estimator = sagemaker.estimator.Estimator(
        image,
        role,
        instance_count=1,
        instance_type='ml.c5.2xlarge',
        volume_size=30,
        max_run=360000,
        input_mode='File',
        enable_sagemaker_metrics=True,
        output_path=s3_output_location,
        hyperparameters={
            'mode':'supervised',
            'epochs':40,
            'min_count':2,
            'learning_rate':lr,
            'vector_dim':10,
            'early_stopping':True,
            'patience':4,
            'min_epochs':5,
            'word_nggrams':2},        
    )
    
    estimator.fit(
        inputs = data_channels,
        job_name=jobname,
        experiment_config = experiment_config,
        wait = False)
    
    print(f'Submitted training job {jobname}')    


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2022-04-13-04-27-10


Submitted training job dbpedia-blazingtext-2022-04-13-04-27-10


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2022-04-13-04-27-11


Submitted training job dbpedia-blazingtext-2022-04-13-04-27-11


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: dbpedia-blazingtext-2022-04-13-04-27-12


Submitted training job dbpedia-blazingtext-2022-04-13-04-27-12
